In [ ]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import os
import time

In [2]:
%%time 

# Abriendo los archivos .csv
folderpath =  r"C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data"
print("Files and directories in a specified path:")
files = []
filenames = []
for filename in os.listdir(folderpath):
    f = os.path.join(folderpath, filename)
    filenames.append(filename)
    files.append(f)
    if os.path.isfile(f):
        print(f)

Files and directories in a specified path:
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_programas_sociales_hogar.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE223132.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE516171.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RE758081.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC84DV.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC91.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC93DVdisciplina.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_REC95.csv
C:\Users\sandr\Dropbox\PUCP\Diplomado\8_Big_Data\Trabajo Final\data\endes_2021_RECH23.csv
Wall time: 16 ms


In [3]:
%%time 

for i in range(len(filenames)):
    filenames[i] = filenames[i][slice(0,-4)]
filenames

Wall time: 0 ns


['endes_2021_programas_sociales_hogar',
 'endes_2021_RE223132',
 'endes_2021_RE516171',
 'endes_2021_RE758081',
 'endes_2021_REC84DV',
 'endes_2021_REC91',
 'endes_2021_REC93DVdisciplina',
 'endes_2021_REC95',
 'endes_2021_RECH23']

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61951,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.81 GiB
Comm: tcp://127.0.0.1:61978,Total threads: 2
Dashboard: http://127.0.0.1:61982/status,Memory: 1.95 GiB
Nanny: tcp://127.0.0.1:61954,


In [5]:
import dask.dataframe as dd

In [6]:
#asignamos nombres a los dataframes
programas_sociales = dd.read_csv(files[0], dtype = str)
nacimiento = dd.read_csv(files[1], dtype = str)
nupcialidad_fecundidad = dd.read_csv(files[2], dtype = str)
sida_condon = dd.read_csv(files[3], dtype = str)
mortalidad_violencia = dd.read_csv(files[4], dtype = str)
MEF = dd.read_csv(files[5], dtype = str) 
disciplina_infantil = dd.read_csv(files[6], dtype = str)
inmunizacion_salud = dd.read_csv(files[7], dtype = str)
vivienda = dd.read_csv(files[8], dtype = str)

In [7]:
%%time 

# identificar bases que tengan CASEID
df_list = [programas_sociales, nacimiento, nupcialidad_fecundidad, sida_condon, mortalidad_violencia, MEF, disciplina_infantil,
          inmunizacion_salud,vivienda]

df_list_names = ["programas_sociales", "nacimiento", "nupcialidad_fecundidad", "sida_condon", "mortalidad_violencia", "MEF", "disciplina_infantil",
          "inmunizacion_salud","vivienda"]

df_CASEID = []

for df, df_name in zip(df_list,df_list_names):
       
    if 'CASEID' in df.columns:
        print(df_name)
        
        df_CASEID.append(df)

nacimiento
nupcialidad_fecundidad
sida_condon
mortalidad_violencia
MEF
disciplina_infantil
inmunizacion_salud
Wall time: 998 µs


In [8]:
%%time 

base_total = nacimiento.merge(nupcialidad_fecundidad, on = ['CASEID', 'ID1'], how ='left').merge(sida_condon, on = ['CASEID', 'ID1'], how ='left').merge(mortalidad_violencia, on = ['CASEID', 'ID1'], how ='left').merge(MEF, on = ['CASEID', 'ID1'], how ='left').merge(disciplina_infantil, on = ['CASEID', 'ID1'], how ='left').merge(inmunizacion_salud, on = ['CASEID', 'ID1'], how ='left')

Wall time: 454 ms


In [11]:
base_total.compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting


KilledWorker: ("('merge_chunk-1c338fd6aa36b48b2e5b3cefc61f3c31', 0)", <WorkerState 'tcp://127.0.0.1:62345', name: 3, status: closed, memory: 0, processing: 1>)

distributed.nanny - WARNING - Restarting worker


In [ ]:
%%time 

df_final = base_total.compute()

distributed.nanny - WARNING - Restarting worker


In [10]:
df_final.dtypes

NameError: name 'df_final' is not defined

In [229]:
%%time 

# Seleccionando las variables de interés 
variables = ['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID',
             'S108N', 'S119', 'S119D', 'V218', 'V213', 'V225',    # carac. personales
             'S45D4', 'S45D5', 'S45P4', 'S45P5', 'S45B0',    # vacunas
             'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2',    # vacunas
             'V501', 'V714', 'D104', 'D106', 'D107', 'D108']

data_final = df_final[variables]
data_final.shape

Wall time: 1.08 s


(43137, 27)

In [238]:
for i in variables:
        if data_final[i].dtype == 'O':
            data_final[i] = data_final[i].str.replace(" ", "")
            
            # Reemplazando los missing values por NaN
            data_final[i] = data_final[i].replace(' ', np.nan)

            # Convirtiendo los datos a valores numéricos
            data_final[i] =  np.floor(pd.to_numeric(data_final[i], errors='coerce'))
            
            # Convirtiendo los datos a valores numéricos
            data_final[i] =  data_final[i].astype('Int64')

In [247]:
data_final.dtypes

Int64Dtype()

In [248]:
data_final = data_final.fillna(0)

In [249]:
for i in variables:
    data_final[i] = data_final[i].astype('Int64')

KeyError: 'ID1'

In [244]:
data_final.dtypes

ID1        float64
SREGION    float64
SPROVIN    float64
SDISTRI    float64
CASEID     float64
S108N      float64
S119       float64
S119D      float64
V218       float64
V213       float64
V225       float64
S45D4      float64
S45D5      float64
S45P4      float64
S45P5      float64
S45B0      float64
S45B1      float64
S45B2      float64
S45B3      float64
S45F1      float64
S45F2      float64
V501       float64
V714       float64
D104       float64
D106       float64
D107       float64
D108       float64
dtype: object

In [241]:
data_final['D104'].astype('Int64')

0           0
1        <NA>
2           1
3        <NA>
4        <NA>
         ... 
43132       0
43133       0
43134    <NA>
43135       0
43136       0
Name: D104, Length: 43137, dtype: Int64

In [177]:
# Agregando los datos para evaluar cuántas mujeres han sido víctimas de violencia en al menos una de sus modalidades
data_final['violencia'] = data_final['D104'] + data_final['D106'] + data_final['D107'] + data_final['D108']

# Contabilizando lo encontrado
data_final['violencia'].value_counts()

0.0    18395
1.0     4372
2.0     2621
3.0     1598
4.0      817
Name: violencia, dtype: Int64

In [178]:
# Creando variable dummy 

data_final['y'] = data_final['violencia']
data_final.loc[(data_final['y'] > 0), 'y'] = 1
data_final['y'].value_counts()

0.0    18395
1.0     9408
Name: y, dtype: Int64

In [181]:
from sklearn.linear_model import *
from sklearn.pipeline import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import *
import pandas as pd

Secuencial

In [183]:
data_final.columns

Index(['ID1', 'SREGION', 'SPROVIN', 'SDISTRI', 'CASEID', 'S108N', 'S119',
       'S119D', 'V218', 'V213', 'V225', 'S45D4', 'S45D5', 'S45P4', 'S45P5',
       'S45B0', 'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2', 'V501', 'V714',
       'D104', 'D106', 'D107', 'D108', 'violencia', 'y'],
      dtype='object')

In [198]:
cols = ['y', 'SREGION', 'SPROVIN', 'SDISTRI', 'S108N', 'S119',
       'S119D', 'V218', 'V213', 'V225', 'S45D4', 'S45D5', 'S45P4', 'S45P5',
       'S45B0', 'S45B1', 'S45B2', 'S45B3', 'S45F1', 'S45F2', 'V501', 'V714']

In [200]:
data_final = data_final[cols]

In [206]:
data_final = data_final[data_final['y'].notna()]

In [209]:
data_final = data_final.fillna(0)

In [210]:
X_train, X_test, y_train, y_test = train_test_split(
    data_final.iloc[:,range(1,len(data_final.columns))],
    data_final['y'],
    test_size=0.3,
    random_state=175904)
print("Number of records: %d"%data_final.shape[0])
data_final[1:5]

Number of records: 27803


,y,SREGION,SPROVIN,SDISTRI,S108N,S119,S119D,V218,V213,V225,...,S45P4,S45P5,S45B0,S45B1,S45B2,S45B3,S45F1,S45F2,V501,V714
2,1.0,3.0,1.0,1.0,0.0,10.0,7.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0
5,0.0,3.0,1.0,1.0,2.0,10.0,7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
6,0.0,3.0,1.0,1.0,3.0,10.0,7.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
7,0.0,3.0,1.0,1.0,2.0,10.0,7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [211]:
%%time 
grid_params = { 
    "lasso0__eps": [1e-3, 1e-4, 1e-5],
    "lasso0__n_alphas": [100, 250, 500]}
lasso0 = LassoCV(max_iter=1000, n_alphas=300, n_jobs=15, random_state=175904)
pipe0  = Pipeline([("lasso0", lasso0)])
grid = GridSearchCV(estimator=pipe0, 
                    param_grid=grid_params, 
                    n_jobs=14, scoring="neg_mean_squared_error")
gridm = grid.fit(X_train, y_train)

Wall time: 7.52 s


In [213]:
print("Score: %f with metric '%s'\nThe best parameters are: %s"%(
    gridm.score(X_test, y_test), 
    grid.scoring,
    gridm.best_params_))

Score: -0.203344 with metric 'neg_mean_squared_error'
The best parameters are: {'lasso0__eps': 0.0001, 'lasso0__n_alphas': 100}


In [214]:
%%time
gridm.predict(X_test)

Wall time: 20.9 ms


array([0.29591755, 0.65275919, 0.23286481, ..., 0.24358405, 0.26818845,
       0.32198567])

Pararelo

In [215]:
import dask.dataframe as dd
import dask.array as da
import dask_ml.model_selection as dms
from dask.distributed import Client
import dask
import numpy as np
from dask import delayed

In [218]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:51217/status,
Dashboard: http://127.0.0.1:51217/status,Workers: 4
Total threads: 8,Total memory: 7.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51218,Workers: 4
Dashboard: http://127.0.0.1:51217/status,Total threads: 8
Started: 7 hours ago,Total memory: 7.81 GiB
Comm: tcp://127.0.0.1:51245,Total threads: 2
Dashboard: http://127.0.0.1:51248/status,Memory: 1.95 GiB
Nanny: tcp://127.0.0.1:51221,


In [219]:
dX_train, dX_test, dy_train, dy_test = (
    dd.from_pandas(X_train, npartitions=3),
    dd.from_pandas(X_test, npartitions=3),
    dd.from_pandas(y_train, npartitions=3),
    dd.from_pandas(y_test, npartitions=3)
)

data_final.head(4)

,y,SREGION,SPROVIN,SDISTRI,S108N,S119,S119D,V218,V213,V225,...,S45P4,S45P5,S45B0,S45B1,S45B2,S45B3,S45F1,S45F2,V501,V714
0,0.0,3.0,1.0,1.0,3.0,10.0,5.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
2,1.0,3.0,1.0,1.0,0.0,10.0,7.0,6.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0
5,0.0,3.0,1.0,1.0,2.0,10.0,7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
6,0.0,3.0,1.0,1.0,3.0,10.0,7.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [220]:
%%time 
grid_params = { 
    "lasso0__eps": [1e-3, 1e-4, 1e-5],
    "lasso0__n_alphas": [100, 250, 500]}

lasso0 = LassoCV(max_iter=1000, n_alphas=300, n_jobs=15, random_state=175904)
pipe0  = Pipeline([("lasso0", lasso0)])
dgrid = dms.GridSearchCV(estimator=pipe0, 
                    param_grid=grid_params, 
                    n_jobs=14, scoring="neg_mean_squared_error")
dgridm = dgrid.fit(dX_train, dy_train)

Wall time: 9.65 s


In [221]:
%%time
y_hat = dX_test.map_partitions(dgridm.predict)
y_hat.compute()

ValueError: Metadata inference failed in `predict`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("float() argument must be a string or a number, not 'NAType'")

Traceback:
---------
  File "C:\Users\sandr\anaconda3\envs\renv\lib\site-packages\dask\dataframe\utils.py", line 177, in raise_on_meta_error
    yield
  File "C:\Users\sandr\anaconda3\envs\renv\lib\site-packages\dask\dataframe\core.py", line 6001, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\Users\sandr\anaconda3\envs\renv\lib\site-packages\sklearn\utils\metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "C:\Users\sandr\anaconda3\envs\renv\lib\site-package